In [ ]:
import rasterio

In [ ]:
from rasterio.mask import mask
import geopandas as gpd
import fiona
import pandas as pd

In [ ]:
ras = 'data/stanford-td754wr4701-geotiff.tiff' # already interpolated but we will treat it as "data"

In [ ]:
shp = 'data/tl_2022_06073_faces.shp'

In [ ]:
gdf = gpd.read_file(shp)

In [ ]:
gdf.shape

In [ ]:
gdf.head()

In [ ]:
county = gdf.dissolve(by='COUNTYFP20')

In [ ]:
county.plot()

In [ ]:
rast = rasterio.open("data/stanford-td754wr4701-geotiff.tiff")

In [ ]:
county = county.to_crs(rast.crs)

In [ ]:
county.plot()

In [ ]:
rast.crs

In [ ]:
coords = gdf.geometry
src = rast
df = county
import matplotlib.pyplot as plt
from rasterio.plot import show

In [ ]:
clipped_array, clipped_transform = mask(dataset=src, shapes=coords, crop=True)

df = df.to_crs(src.crs)
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": clipped_array.shape[1],
                 "width": clipped_array.shape[2],
                 "transform": clipped_transform})
out_tif= "clipped_example.tif"
with rasterio.open(out_tif, "w", **out_meta) as dest:
    dest.write(clipped_array)
    
clipped = rasterio.open(out_tif)
fig, ax = plt.subplots(figsize=(8, 6))
p1 = df.plot(color=None,facecolor='none',edgecolor='red',linewidth = 2,ax=ax)
show(clipped, ax=ax)
ax.axis('off');

In [ ]:
clipped

In [ ]:
import rioxarray

In [ ]:
d = rioxarray.open_rasterio("clipped_example.tif")

In [ ]:
d

In [ ]:
d.plot()

In [ ]:
d.values.max()

In [ ]:
d.plot()

In [ ]:
type(d)

In [ ]:
d.dims

In [ ]:
d.values.mean()

In [ ]:
import numpy


In [ ]:
numpy.median(d.values)

In [ ]:
d.values.shape

In [ ]:
d.plot()

In [ ]:
d.plot.hist()

In [ ]:
type(d)

In [ ]:
import rasterstats

In [ ]:
gdf.head()

In [ ]:
tracts = gdf.dissolve(by='TRACTCE20')

In [ ]:
tracts.shape

In [ ]:
tracts.plot()

In [ ]:
from rasterstats import zonal_stats
tstats = zonal_stats(tracts, "clipped_example.tif",
            stats="count min mean max median")

#elevations2 = zonal_stats(
#    sd_tracts.to_crs(dem.rio.crs),  # Geotable with zones
#    "../data/nasadem/nasadem_sd.tif",  # Path to surface file
#)
#elevations2 = pandas.DataFrame(elevations2)

In [ ]:
tstats[:5]

In [ ]:
tstats = pd.DataFrame(tstats)

In [ ]:
tstats.head()

In [ ]:
tstats.shape

In [ ]:
tracts.shape

In [ ]:
tracts['mean'] = tstats['mean'].values
tracts.plot(column='mean', legend=True);

In [ ]:
tracts['median'] = tstats['median'].values
tracts.plot(column='median', legend=True);

In [ ]:
tracts['range'] = tstats['max'].values - tstats['min'].values
tracts.plot(column='range', legend=True);